In [11]:
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection

In [12]:

connections.connect("default", host="localhost", port="19530")

In [13]:
collection_name = "IPL_collection"
query_text = "What is the capacity of wankhede stadium?"
model = SentenceTransformer("all-MiniLM-L6-v2")
collection = Collection(name=collection_name)

query_vector = model.encode([query_text])[0]
collection.load()
search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}

results = collection.search(
        data=[query_vector],
        anns_field="vector",
        param=search_params,
        limit=15,
        output_fields=["text", "source"]
    )

for result in results[0]:
        print(f"\nScore: {result.score:.4f}")
        print(f"Text: {result.entity.get('text')}")
        print(f"Source: {result.entity.get('source')}")


Score: 0.5729
Text: cket Association of
Bengal, it was the largest cricket stadium in India and had a seating capacity of over
90,000.[28]  In  2011,  the  stadium  was  renovated  to  meet  the  standards  set  by  the
International  Cricket  Council  for  the  2011  Cricket  World  Cup;  reducing  its  capacity  to
around 68,000. The renovated stadium includes a new clubhouse and players' facilities,
upgrading the exterior wall, cladding the existing roof structure with a new metal skin,
and general infrastructur
Source: IPL_Teams.pdf

Score: 0.5065
Text: rld Cup. Mumbai Indians won six out of
the seven matches at the Brabourne Stadium that season.
Mumbai  Indians  now  play  their  home  games  at  Wankhede  Stadium  in  Mumbai.  The
 stadium is named after former BCCI President S. K. Wankhede. The stadium is owned
by the Mumbai Cricket Association and has a seating capacity of 33,108.
The motto of the team is "Duniya Hila Denge Hum...", which translates to We will rock
the world.[

In [14]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

reranked_results = sorted(results[0], key=lambda x: x.distance)
top_n = 10
top_chunks = [res.entity.get("text") for res in reranked_results[:top_n]]
context = "\n\n".join(top_chunks)

print(context)

ver  the  captaincy  change  and  the  team's
disjointed performance, Mumbai finished the season in last place with only 4 wins.
The Mumbai Indians played home games at the DY Patil Stadium in Navi Mumbai for the
first two 
IPL  seasons.  In  the  third  season,  in  2010,  they  played  all  seven  home  games  at  the
Brabourne Stadium while the Wankhede Stadium underwent renovation to host group
matches and the final of the 2011 ICC Cricket World Cup. Mumbai Indians won six out of
the seven m

over-hauled
the Jawaharlal Nehru Stadium to cater for hosting IPL matches.[24]
Federal  Bank,  a  major  Indian  commercial  bank  in  the  private  sector,  with  its
headquarters  in  Aluva,  Kerala,  was  the principal sponsor of Kochi Tuskers Kerala.[25]
Lotto, an Italian sports goods manufacturer, was the apparel sponsor,[26] V-Guard, an
electronics  company,  and  AVT,  a  tea  brand,  were  the  associate  sponsors  with  Elite
Foods, Parinee Developers and Anchor Earth are the other ma

In [15]:
question = "What is the capacity of wankhede stadium?"

In [16]:
template = PromptTemplate.from_template(
    """You are a helpful assistant.

    Use the following context to answer the question. Only return the specific answer, be brief, and do not repeat the context.

    Context:
    {context}

    Question: {question}
    Answer:"""
)

In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

In [18]:
chain = template | llm | parser
answer = chain.invoke({"context": context, "question": question})

In [19]:
print("\nFinal Answer:\n", answer)


Final Answer:
 33,108
